## DSN  : Predicitve Model Notebook [Using Light GBM]
**Author**:🧕🏿 Hasanat Owoseni \
**Date** : 14th October, 2019

In [28]:
import numpy as np
import pandas as pd
%matplotlib inline

In [29]:
test_df = pd.read_csv('cleaned_test.csv')
train_df = pd.read_csv('cleaned_train.csv')

In [30]:
test_df.drop(['Unnamed: 0'], axis = 1, inplace = True)
train_df.drop(['Unnamed: 0'], axis = 1, inplace = True)
train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38312 entries, 0 to 38311
Data columns (total 30 columns):
employeeno                                                38312 non-null object
trainings_attended                                        38312 non-null int64
last_performance_score                                    38312 non-null float64
targets_met                                               38312 non-null int64
previous_award                                            38312 non-null int64
training_score_average                                    38312 non-null int64
no_of_previous_employers                                  38312 non-null int64
promoted_or_not                                           38312 non-null int64
age                                                       38312 non-null int64
working_for                                               38312 non-null int64
is_female                                                 38312 non-null int64
schooled_abroad    

In [31]:
columns = [column for column in test_df.columns if column != 'employeeno']
test_x = test_df[columns].values.astype('float')

test_empid = test_df['employeeno']
#what test x should be 
s_test_x = test_df.loc[: ,'trainings_attended':].values.astype('float')

In [32]:
columns = [column for column in train_df.columns if column != ('promoted_or_not' ) and column !=( 'employeeno')]

X = train_df[columns].values.astype('float')
s_train = train_df[columns]

y = train_df['promoted_or_not']
empid = train_df['employeeno']

In [33]:
print(test_x.shape)
print(X.shape)
print(s_test_x.shape)
s_train.info()

(16496, 28)
(38312, 28)
(16496, 28)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38312 entries, 0 to 38311
Data columns (total 28 columns):
trainings_attended                                        38312 non-null int64
last_performance_score                                    38312 non-null float64
targets_met                                               38312 non-null int64
previous_award                                            38312 non-null int64
training_score_average                                    38312 non-null int64
no_of_previous_employers                                  38312 non-null int64
age                                                       38312 non-null int64
working_for                                               38312 non-null int64
is_female                                                 38312 non-null int64
schooled_abroad                                           38312 non-null int64
is_married                                                38312

In [34]:
#train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(30649, 28) (30649,)
(7663, 28) (7663,)


--------------------- 

In [35]:
import lightgbm as lgb

In [36]:
train_data = lgb.Dataset(X_train, label = y_train)

In [52]:
#setting parameters for lightgbm
param = {'num_leaves':200, 'objective':'binary', 'max_depth':7, 'learning_rate':0.15,'max_bin':500}
param['metric'] = ['auc', 'binary_logloss']

In [53]:
#training our model using light gbm
num_round=50
lgbm=lgb.train(param,train_data,num_round)
#(param, train_data , num_round)

In [47]:
#fitting the lgbm model:


In [54]:
ypred2 = lgbm.predict(test_x)
ypred2[0:5]  # showing first 5 predictions
y_test[0:5]

18116    0
25421    0
10129    0
18814    0
27359    0
Name: promoted_or_not, dtype: int64

In [55]:
for i in range(0,16496):
    if ypred2[i]>=.5:       # setting threshold to .5
        ypred2[i]=1.0
    else:  
        ypred2[i]=0.0

from sklearn.metrics import accuracy_score

In [56]:
ypred2[0:5]  # showing first 5 predictions

array([0., 0., 0., 0., 0.])

In [57]:
df_submission = pd.DataFrame({'employeeno':test_df['employeeno'], 'promoted_or_not':ypred2.astype('int64')})
df_submission.to_csv('second2.csv', index = False)

In [26]:
from sklearn.metrics import roc_auc_score

In [27]:
auc_lgbm

NameError: name 'auc_lgbm' is not defined